In [80]:
import itertools
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import datasets, svm
import numpy as np
import pennylane as qml
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import os
import covalent as ct

In [ ]:
batch_executor = ct.executor.AWSBatchExecutor(
    profile="aq",
    region="us-east-1",
    batch_execution_role_name = "ae-batch-aws-demo-task-execution-role",
    batch_job_definition_name = "ae-batch-aws-demo-job-definition",
    batch_job_log_group_name = "ae-batch-aws-demo-log-group",
    batch_job_role_name = "ae-batch-aws-demo-job-role",
    batch_queue = "ae-batch-aws-demo-queue",
    s3_bucket_name = "ae-batch-aws-demo-job-resources"
)

batch_deps_pip=ct.DepsPip(packages=["scikit-learn","torch","torchvision"])

In [82]:
lambda_executor = ct.executor.AWSLambdaExecutor(
    profile="aq",
    region="us-east-1",
    function_name = "ae-lambda-aws-demo-lambda-fn",
    s3_bucket_name = "ae-lambda-aws-demo-covalent-artifact-bucket",
)

lambda_deps_pip = ct.DepsPip(packages=["scikit-learn"])

In [86]:
braket_executor = ct.executor.BraketExecutor(
    profile="aq",
    region="us-east-1",
    braket_job_execution_role_name = "amazon-braket-ae-aws-demo-role",
    ecr_image_uri = "348041629502.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-ae-aws-demo-base-executor-repo:latest",
    s3_bucket_name = "amazon-braket-ae-aws-demo-bucket",
    time_limit=60*60
)

braket_deps_pip = ct.DepsPip(packages=["scikit-learn"])

In [88]:
# Low compute
@ct.electron
def get_feature_combinations(num_features):
    features = []
    for length in range(num_features):
        for comb in itertools.combinations(range(num_features), length):
            features.append(comb)
    return features[1:]


# Sent to lambda
@ct.electron(
    executor=lambda_executor,
    deps_pip=lambda_deps_pip
)
def fit_models_calculate_accuracy(data, model, features_to_select):
    X_train, X_test, y_train, y_test = data
    X_train = X_train[:, features_to_select]
    model.fit(X_train, y_train)
    return model, model.score(X_test[:, features_to_select], y_test)


# Low compute
@ct.electron
def load_data(reduce_dims=12, num_data_points=None):
    X, y = datasets.load_breast_cancer(return_X_y=True)
    if num_data_points:
        X = X[:num_data_points]
        y = y[:num_data_points]
    X = PCA(n_components=reduce_dims).fit_transform(X)
    data = train_test_split(X, y, test_size=0.5)
    return data


# Low compute
@ct.electron
def select_best_model(models_and_accuracies, feature_combinations):
    best_accuracy = 0
    best_model = None
    best_position = 0
    for i, (model, accuracy) in enumerate(models_and_accuracies):
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_position = i
    return best_model, feature_combinations[best_position]


@ct.electron
def find_best_features(data, model, feature_combinations):
    models_and_accuracies = []
    for feature_list in feature_combinations:
        models_and_accuracies_tmp = fit_models_calculate_accuracy(data, model, feature_list)
        models_and_accuracies.append(models_and_accuracies_tmp)
    return select_best_model(models_and_accuracies, feature_combinations)


# Low compute
@ct.electron
def plot_confusion_matrix(model, data, feature_list):
    X_train, X_test, y_train, y_test = data
    X_test = X_test[:, feature_list]
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

    predictions = model.predict(X_test)
    cm = confusion_matrix(y_test, predictions, labels=model.classes_)
    return ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)


# Braket executor
@ct.electron(
    executor=braket_executor,
    deps_pip=braket_deps_pip
)
def do_quantum_svm(data, features, ansatz):
    device_arn = os.environ["AMZN_BRAKET_DEVICE_ARN"]
    s3_bucket = os.environ["AMZN_BRAKET_OUT_S3_BUCKET"]
    s3_task_dir = os.environ["AMZN_BRAKET_TASK_RESULTS_S3_URI"].split(s3_bucket)[1]

    device = qml.device(
        "braket.aws.qubit",
        device_arn=device_arn,
        s3_destination_folder=(s3_bucket, s3_task_dir),
        wires=features
    )
    
    @qml.qnode(device)
    def kernel(x1, x2):
        ansatz(x1, wires=features)
        qml.adjoint(ansatz)(x2, wires=features)
        return qml.expval(qml.Projector(range(len(features)), features))

    quantum_kernel = lambda x1, x2: np.array([[kernel(i, j).numpy() for i in x1] for j in x2])
    model = svm.SVC(kernel=quantum_kernel)
    X_train, X_test, y_train, y_test = data
    X_train = X_train[:, features]
    X_test = X_test[:, features]

    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)

    return model, score


# low compute
@ct.electron
def get_data(data, features, batch_size=4):
    class Data(Dataset):
        def __init__(self, X_train, y_train):
            self.X = torch.from_numpy(X_train.astype(np.float32))
            self.y = torch.from_numpy(y_train).type(torch.LongTensor)
            self.len = self.X.shape[0]

        def __getitem__(self, index):
            return self.X[index], self.y[index]

        def __len__(self):
            return self.len

    X_train, X_test, y_train, y_test = data
    X_test = X_test[:, features]
    X_train = X_train[:, features]
    traindata = Data(X_train, y_train)
    trainloader = DataLoader(traindata, batch_size=batch_size, shuffle=True)
    return trainloader

# Batch compute
@ct.electron(
    executor=batch_executor,
    deps_pip=batch_deps_pip
)
def train_dnn(data, features, trainloader, hidden_layers=10, epochs=10, layers_size=5):
    input_dim = len(features)
    output_dim = trainloader.dataset.y.unique().shape[0]

    def get_net(hidden_layers):
        layers = [nn.Linear(input_dim, layers_size)]
        for _ in range(hidden_layers):
            layers.append(nn.Linear(layers_size, layers_size))
            layers.append(nn.Dropout(p=0.2))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(layers_size, output_dim),)
        return layers

    class Network(nn.Module):
        def __init__(self):
            super(Network, self).__init__()
            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(*get_net(hidden_layers))

        def forward(self, x):
            x = self.flatten(x)
            x = self.linear_relu_stack(x)
            return x

    clf = Network()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(clf.parameters(), lr=0.05)

    losses = []
    for _ in range(epochs):
        running_loss = 0.0
        for i, data_train in enumerate(trainloader, 0):
            inputs, labels = data_train
            optimizer.zero_grad()
            outputs = clf(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            losses.append(running_loss)

    _, X_test, _, y_test = data
    X_test = X_test[:, features]
    _, prediction = torch.max(clf(torch.Tensor(X_test)), 1)
    accuracy = (y_test == prediction.numpy()).sum() / y_test.shape[0]
    return clf, accuracy


@ct.electron
def compare_classical_quantum(classical_accuracy, quantum_accuracy):
    plt.switch_backend("Agg")
    fig, ax = plt.subplots()

    sns.set_theme(style="whitegrid", palette="pastel")
    data = {"x": ["Classical", "Quantum"], "y": [classical_accuracy, quantum_accuracy]}
    sns.barplot(x="x", y="y", data=data, ax=ax)
    ax.bar_label(ax.containers[0])
    ax.set_ylabel("Accuracy")
    plt.close()
    return fig


@ct.lattice
def my_experiment(
    num_data_points,
    feature_reduction_model,
    reduce_dims,
    quantum_ansatz,
    classical_hidden_layers,
    classical_layers_size,
    classical_epochs,
):
    data = load_data(reduce_dims=reduce_dims, num_data_points=num_data_points)
    feature_combinations = get_feature_combinations(reduce_dims)
    best_model, best_features = find_best_features(
        data, feature_reduction_model, feature_combinations
    )
    plot_confusion_matrix(best_model, data, best_features)
    quantum_svm_model, quantum_score = do_quantum_svm(data, best_features, quantum_ansatz)
    train_loader = get_data(data, best_features)
    classical_dnn, classical_score = train_dnn(
        data,
        best_features,
        trainloader=train_loader,
        hidden_layers=classical_hidden_layers,
        layers_size=classical_layers_size,
        epochs=classical_epochs,
    )
    compare_classical_quantum(classical_score, quantum_score)
    return quantum_svm_model, quantum_score, classical_dnn, classical_score


# number of points in dataset (increasing this will increase SVM memory as well as quantum SVM kernel computation time)
num_data_points = 200  # None implies all data ~ 500
# Controls the number of features which increases the number of lambdas
reduce_dims = 3
feature_reduction_model = svm.SVC()
quantum_ansatz = qml.AngleEmbedding

# Controls calssical batch processing power
classical_hidden_layers = 3
classical_layers_size = 3
classical_epochs = 10

runid = ct.dispatch(my_experiment)(
    num_data_points=num_data_points,
    feature_reduction_model=feature_reduction_model,
    reduce_dims=reduce_dims,
    quantum_ansatz=quantum_ansatz,
    classical_hidden_layers=classical_hidden_layers,
    classical_layers_size=classical_layers_size,
    classical_epochs=classical_epochs,
)
print(runid)

b116242a-f0f0-4d66-a18d-3ae09d9eba04
